In [ ]:
require 'nn'
require 'rnn'
require 'optim'

In [ ]:
nepochs = 250
learning_rate = 0.1
gamma =0.2
cuts =4
base_explore_rate = 0.1
delta = 1./(nepochs/cuts) 

SKIP = 1
SELECT = 2

vocabSize = 16
embeddingSize = 64

torch.manualSeed(420)
math.randomseed(420)

sentenceLookup = nn.Sequential():add(
    nn.LookupTableMaskZero(vocabSize, embeddingSize)):add(
    nn.Sum(2, 3, false))

queryLookup = sentenceLookup:clone() --"weight", "gradWeight")
summaryLookup = sentenceLookup:clone() --"weight", "gradWeight")

model = nn.Sequential():add(
    nn.ParallelTable():add(
        sentenceLookup):add(
        queryLookup):add(
        summaryLookup)):add(
    nn.JoinTable(2)):add(
    nn.Tanh()):add(
    nn.Linear(embeddingSize * 3, 2)) --:add(
    --nn.Tanh()):add(
    --nn.Linear(embeddingSize, 2))
criterion = nn.MSECriterion()
params, gradParams = model:getParameters()

function buildSummary(actions, sentences, buffer)
    buffer:zero()

     bufferSize = buffer:size(2)
     actionsSize = actions:size(1)
     sentencesSize = sentences:size(2)

     mask1 = torch.eq(actions:select(2,2), 1):view(actionsSize, 1):expand(
        actionsSize, sentencesSize)
     allTokens = sentences:maskedSelect(mask1)
     mask2 = torch.gt(allTokens,0)
     allTokens = allTokens:maskedSelect(mask2)

    if allTokens:dim() > 0 then
         copySize = math.min(bufferSize, allTokens:size(1))

        buffer[1]:narrow(1, bufferSize - copySize + 1, copySize):copy(
            allTokens:narrow(1, allTokens:size(1) - copySize + 1, copySize))
    end
    return buffer
end

function buildTokenCounts(summary)
    counts = {}
    for i=1,summary:size(2) do
        if summary[1][i] > 0 then
             token = summary[1][i]
            if counts[token] == nil then
                counts[token] = 1
            else
                counts[token] = counts[token] + 1
            end
        end
    end
    return counts
end

function rougeScores(genSummary, refSummary)
     genTotal = 0
     refTotal = 0
     intersection = 0
    for k, refCount in pairs(refSummary) do
         genCount = genSummary[k]
        if genCount == nil then genCount = 0 end
        intersection = intersection + math.min(refCount, genCount)
        refTotal = refTotal + refCount
    end
    for k,genCount in pairs(genSummary) do
        genTotal = genTotal + genCount
    end

    if genTotal == 0 then 
        genTotal = 1 
    end
     recall = intersection / refTotal
     prec = intersection / genTotal
    if recall > 0 and prec > 0 then
        f1 = 2 * recall * prec / (recall + prec)
    else 
        f1 = 0
    end
    return recall, prec, f1
end

In [ ]:
optimParams = {
    learningRate = learning_rate,
}

maxSummarySize = 36
epsilon = 1.0
query = torch.LongTensor{{0, 1, 4, 3}}
sentenceStream = torch.Tensor{{0, 1, 3, 4}, 
                                {7, 6, 5 ,8}, 
                                {0, 2, 4, 3}, 
                                {7, 5, 8, 6}, 
                                {1, 4, 3, 2}, 
                                {13, 14, 15, 16}}

refSummary = torch.Tensor{{1,3,4,2,4,3,1,4,3,2,9,10,12,11}}
refCounts = buildTokenCounts(refSummary)


streamSize = sentenceStream:size(1)
bestActions = torch.ByteTensor{{0,1},{1,0},{0,1},{1,0},{0,1},{1,0}}


buffer = torch.Tensor(1, maxSummarySize):zero()
bestSummary = buildSummary(
    bestActions:narrow(1, 1, 6), 
    sentenceStream:narrow(1, 1, 6),
    buffer:narrow(1, 1, 1)
    )

generatedCounts = buildTokenCounts(bestSummary) 
bestrecall, bestprec, bestf1 = rougeScores(generatedCounts, refCounts)
print(string.format("TRUE {RECALL = %.6f, PREC = %.6f, F1 = %.6f}", bestrecall, bestprec, bestf1))

In [ ]:
epoch = 1

In [ ]:
actions = torch.ByteTensor(streamSize,2):fill(0)
exploreDraws = torch.Tensor(streamSize)
summaryBuffer = torch.LongTensor(streamSize + 1, maxSummarySize):zero()
qValues = torch.Tensor(streamSize, 2):zero()
rouge = torch.Tensor(streamSize + 1):zero()
rougebp = torch.Tensor(streamSize + 1):zero()

rouge[1] = 0
exploreDraws:uniform(0, 1)
summary = summaryBuffer:zero():narrow(1,1,1)

In [ ]:
for i=1, streamSize do
    sentence = sentenceStream:narrow(1, i, 1)
    qValues[i]:copy(model:forward({sentence, query, summary}))
    
    if exploreDraws[i] <= epsilon then
        actions[i][torch.random(SKIP, SELECT)] = 1
    else 
        if qValues[i][SKIP] > qValues[i][SELECT] then
            actions[i][SKIP] = 1
        else
            actions[i][SELECT] = 1
        end
    end
    summary = buildSummary(
        actions:narrow(1, 1, i), 
        sentenceStream:narrow(1, 1, i),
        summaryBuffer:narrow(1, i + 1, 1)
        )

    generatedCounts = buildTokenCounts(summary) 
    recall, prec, f1 = rougeScores(generatedCounts, refCounts)
    rouge[i + 1] = f1
    rougebp[i] = f1
end

max, argmax = torch.max(qValues, 2)
rewards0 = rouge:narrow(1,2, streamSize) - rouge:narrow(1,1, streamSize) 
rewards = rewards0 + gamma * rewards0:narrow(1, 2, streamSize-1):resize(streamSize)

querySize = query:size(2)
summaryBatch = summaryBuffer:narrow(1, 1, streamSize)
queryBatch = query:view(1, querySize):expand(streamSize, querySize) 

input = {sentenceStream, queryBatch, summaryBatch}

In [ ]:
function feval(params)
    gradParams:zero()
     predQ = model:forward(input)
     maskLayer = nn.MaskedSelect()
     predQOnActions = maskLayer:forward({predQ, actions})
     loss = criterion:forward(predQOnActions, reward)
     gradOutput = criterion(predQOnActions, reward)
     gradMaskLayer = maskLayer:backward({predQ, actions}, gradOutput)
    model:backward(input, gradMaskLayer[1])
    return loss, gradParams    
end

 _, loss = optim.adam(feval, params, optimParams)

In [ ]:
out = string.format("%i; %.3f;%.6f;%.6f; {min=%.3f, max=%.3f}; {min=%.3f, max=%.3f}; {%i,%i,%i,%i,%i,%i,%i,%i,%i,%i,%i,%i}\n", 
    epoch, epsilon, loss[1], rouge[streamSize + 1],
    reward:min(), reward:max(),
    qValues:min(), qValues:max(),
    actions[1][1], 
    actions[1][2], 
    actions[2][1], 
    actions[2][2], 
    actions[3][1], 
    actions[3][2], 
    actions[4][1],
    actions[4][2],
    actions[5][1], 
    actions[5][2], 
    actions[6][1], 
    actions[6][2] 
    )


In [ ]:
out

In [ ]:
nepochs

# Trying this separately

In [ ]:
require 'nn'
require 'rnn'
require 'optim'

In [ ]:
nepochs = 1000
learning_rate = 1e-6
gamma =0.2
cuts =4
base_explore_rate = 0.1 
delta = 1./(nepochs/cuts) 

SKIP = 1
SELECT = 2

vocabSize = 16
embeddingSize = 64

torch.manualSeed(420)
math.randomseed(420)

sentenceLookup = nn.Sequential():add(
    nn.LookupTableMaskZero(vocabSize, embeddingSize)):add(
    nn.Sum(2, 3, false))

queryLookup = sentenceLookup:clone() --"weight", "gradWeight")
summaryLookup = sentenceLookup:clone() --"weight", "gradWeight")

model = nn.Sequential():add(
    nn.ParallelTable():add(
        sentenceLookup):add(
        queryLookup):add(
        summaryLookup)):add(
    nn.JoinTable(2)):add(
    nn.Tanh()):add(
    nn.Linear(embeddingSize * 3, 2)) --:add(
    --nn.Tanh()):add(
    --nn.Linear(embeddingSize, 2))
criterion = nn.MSECriterion()
params, gradParams = model:getParameters()

function buildSummary(actions, sentences, buffer)
    buffer:zero()

     bufferSize = buffer:size(2)
     actionsSize = actions:size(1)
     sentencesSize = sentences:size(2)

     mask1 = torch.eq(actions:select(2,2), 1):view(actionsSize, 1):expand(
        actionsSize, sentencesSize)
     allTokens = sentences:maskedSelect(mask1)
     mask2 = torch.gt(allTokens,0)
     allTokens = allTokens:maskedSelect(mask2)

    if allTokens:dim() > 0 then
         copySize = math.min(bufferSize, allTokens:size(1))

        buffer[1]:narrow(1, bufferSize - copySize + 1, copySize):copy(
            allTokens:narrow(1, allTokens:size(1) - copySize + 1, copySize))
    end
    return buffer
end

function buildTokenCounts(summary)
    counts = {}
    for i=1,summary:size(2) do
        if summary[1][i] > 0 then
             token = summary[1][i]
            if counts[token] == nil then
                counts[token] = 1
            else
                counts[token] = counts[token] + 1
            end
        end
    end
    return counts
end

function rougeScores(genSummary, refSummary)
     genTotal = 0
     refTotal = 0
     intersection = 0
    for k, refCount in pairs(refSummary) do
         genCount = genSummary[k]
        if genCount == nil then genCount = 0 end
        intersection = intersection + math.min(refCount, genCount)
        refTotal = refTotal + refCount
    end
    for k,genCount in pairs(genSummary) do
        genTotal = genTotal + genCount
    end

    if genTotal == 0 then 
        genTotal = 1 
    end
     recall = intersection / refTotal
     prec = intersection / genTotal
    if recall > 0 and prec > 0 then
        f1 = 2 * recall * prec / (recall + prec)
    else 
        f1 = 0
    end
    return recall, prec, f1
end

In [ ]:
optimParams = {
    learningRate = learning_rate,
}

maxSummarySize = 36
epsilon = 1.0
query = torch.LongTensor{{0, 1, 4, 3}}
sentenceStream = torch.Tensor{{0, 1, 3, 4}, 
                                {7, 6, 5 ,8}, 
                                {0, 2, 4, 3}, 
                                {7, 5, 8, 6}, 
                                {1, 4, 3, 2}, 
                                {13, 14, 15, 16}}

refSummary = torch.Tensor{{1,3,4,2,4,3,1,4,3,2,9,10,12,11}}
refCounts = buildTokenCounts(refSummary)


streamSize = sentenceStream:size(1)
bestActions = torch.ByteTensor{{0,1},{1,0},{0,1},{1,0},{0,1},{1,0}}


buffer = torch.Tensor(1, maxSummarySize):zero()
bestSummary = buildSummary(
    bestActions:narrow(1, 1, 6), 
    sentenceStream:narrow(1, 1, 6),
    buffer:narrow(1, 1, 1)
    )

generatedCounts = buildTokenCounts(bestSummary) 
bestrecall, bestprec, bestf1 = rougeScores(generatedCounts, refCounts)
print(string.format("TRUE {RECALL = %.6f, PREC = %.6f, F1 = %.6f}", bestrecall, bestprec, bestf1))

In [ ]:
tmp0 = {}
tmp1 = {}
tmp2 = {}
for epoch=1,nepochs do
    actions = torch.ByteTensor(streamSize,2):fill(0)
    exploreDraws = torch.Tensor(streamSize)
    summaryBuffer = torch.LongTensor(streamSize + 1, maxSummarySize):zero()
    qValues = torch.Tensor(streamSize, 2):zero()
    rouge = torch.Tensor(streamSize + 1):zero()

    rouge[1] = 1
    exploreDraws:uniform(0, 1)

    summary = summaryBuffer:zero():narrow(1,1,1)
    for i=1, streamSize do
        --- the i extracts individual sentences from the stream
         sentence = sentenceStream:narrow(1, i, 1)
        qValues[i]:copy(model:forward({sentence, query, summary}))

        if exploreDraws[i] <= epsilon then
            actions[i][torch.random(SKIP, SELECT)] = 1
        else 
            if qValues[i][SKIP] > qValues[i][SELECT] then
                actions[i][SKIP] = 1
            else
                actions[i][SELECT] = 1
            end
        end

        summary = buildSummary(
            actions:narrow(1, 1, i), 
            sentenceStream:narrow(1, 1, i),
            summaryBuffer:narrow(1, i + 1, 1)
            )

         generatedCounts = buildTokenCounts(summary) 
         recall, prec, f1 = rougeScores(generatedCounts, refCounts)
        rouge[i + 1] = f1
    end

     max, argmax = torch.max(qValues, 2)
    
     reward0 = rouge:narrow(1,2, streamSize) - rouge:narrow(1,1, streamSize)
     reward_tp1 = gamma * reward0:narrow(1, 2, streamSize - 1):resize(streamSize)
     reward_tp1[reward_tp1:ne(reward_tp1)] = 0
     reward = reward0 + reward_tp1
    
     tmp0[epoch] = reward0
     tmp1[epoch] = reward_tp1
     tmp2[epoch] = reward
    
     querySize = query:size(2)
     summaryBatch = summaryBuffer:narrow(1, 1, streamSize)
     queryBatch = query:view(1, querySize):expand(streamSize, querySize) 

     input = {sentenceStream, queryBatch, summaryBatch}
    if epoch == 1 then
        print(input)
    end

     function feval(params)
        gradParams:zero()
         predQ = model:forward(input)
         maskLayer = nn.MaskedSelect()
         predQOnActions = maskLayer:forward({predQ, actions})

         loss = criterion:forward(predQOnActions, reward)
         gradOutput = criterion(predQOnActions, reward)
         gradMaskLayer = maskLayer:backward({predQ, actions}, gradOutput)
        model:backward(input, gradMaskLayer[1])
        return loss, gradParams    
    end

     _, loss = optim.adam(feval, params, optimParams)
    out = string.format("%i; %.3f;%.6f;%.6f; {min=%.3f, max=%.3f}; {min=%.3f, max=%.3f}; {%i,%i,%i,%i,%i,%i,%i,%i,%i,%i,%i,%i}\n", 
            epoch, epsilon, loss[1], rouge[streamSize + 1],
            reward:min(), reward:max(),
            qValues:min(), qValues:max(),
            actions[1][1], 
            actions[1][2], 
            actions[2][1], 
            actions[2][2], 
            actions[3][1], 
            actions[3][2], 
            actions[4][1],
            actions[4][2],
            actions[5][1], 
            actions[5][2], 
            actions[6][1], 
            actions[6][2] 
        )
    print(out)

    if (epsilon - delta) <= base_explore_rate then
        epsilon = base_explore_rate
    else 
        epsilon = epsilon - delta
    end
end


In [ ]:
torch.clamp(tmp0[360], -0.7, 0.7)

In [ ]:
tmp1[360]

In [ ]:
tmp1[1]

In [ ]:
nn.Threshold(0.06, 0):forward(tmp1[1])

In [ ]:
nn.Threshold(-.05, 0):forward(tmp0[360])

In [ ]:
tmp1[360]

In [ ]:
tmp2[360]

In [ ]:
tmp1[6][tmp1[6]:ne(tmp1[6])] = 0

In [ ]:
tmp0[6]:narrow(1, 2, streamSize-1):resize(streamSize)

In [ ]:
tmp0[6]:narrow(1, 2, streamSize-1):resize(streamSize)

In [ ]:
tmp0[6] + gamma * tmp0[6]:narrow(1, 2, streamSize-1):resize(streamSize)

In [ ]:
tmp1[6]

In [ ]:
tmp0[242]

In [ ]:
--- If tmp0[242] < thresh, out else tmp0[242]
thresh = -0.7
out = 99
print(nn.Threshold(thresh, out):forward(tmp0[242]))

In [ ]:
--- If tmp0[242] < thresh, out else tmp0[242]
thresh = -0.25
out = 99
print(nn.Threshold(thresh, out):forward(-tmp0[242]))

In [ ]:
nn.Threshold(0.01, -0.001):forward(tmp0[242])

In [ ]:
nn.Threshold(0.1, 0.01):forward(nn.Threshold(-0.01, -0.01):forward(tmp0[242]))

In [ ]:
for i = 1, 500 do
    print(tmp0[i] + tmp1[i])
end

In [ ]:
sentenceStream

In [ ]:
tst = torch.cat(sentenceStream, sentenceStream, 1)

In [ ]:
tst

In [ ]:
local dl = require 'dataload'

inputs = torch.range(1,5)
targets = torch.range(1,5)

dataloader = dl.TensorLoader(inputs, targets)

local i = 1
for k, inputs, targets in dataloader:sampleiter(2,6) do
   print(string.format("batch %d, nsampled = %d; inputs = %.i", i, k, inputs[1]))
--    print(string.format("inputs:\n%stargets:\n%s", inputs, targets))
   i = i + 1
end

In [8]:
inputs = torch.randn(5, 1)
targets = torch.LongTensor(10):random(1,10):resize(5,2)
dataloader = dl.TensorLoader(inputs, targets)

In [27]:
i = 1
for k, input, target in dataloader:sampleiter(5, 20) do
    print(string.format("batch %d, nsampled = %d; target {i, i}", i, k))
    print(target)
   i = i + 1
end

batch 1, nsampled = 5; target {i, i}	
 10   3
 10   5
 10   3
  6   2
 10   3
[torch.LongTensor of size 5x2]

batch 2, nsampled = 10; target {i, i}	
  1   6
 10   3
  6   2
 10   5
  3   3
[torch.LongTensor of size 5x2]

batch 3, nsampled = 15; target {i, i}	
  1   6
 10   5
 10   5
  1   6
  6   2
[torch.LongTensor of size 5x2]

batch 4, nsampled = 20; target {i, i}	
  3   3
 10   3
  1   6
 10   5
  6   2
[torch.LongTensor of size 5x2]



In [6]:
dataloader

dl.TensorLoader
{
  inputs : DoubleTensor - size: 5x1
  _indices : LongTensor - size: 2
  gccount : 3
  targets : LongTensor - size: 5
  gcdelay : 200
}


In [1]:
dl = require 'dataload'

inputs = torch.LongTensor({1,2, 3, 5}, {1, 2, 3}, {4, 5, 7}, {7, 8 ,8, 3, 2}, {1,2,3,4})
targets = torch.LongTensor({1,2, 3, 5}, {1, 2, 3}, {4, 5, 7}, {7, 8 ,8, 3, 2}, {1,2,3,4})
actions = torch.round(torch.rand(5))

In [2]:
dataloader = dl.TensorLoader(inputs, actions)

...eo/torch/install/share/lua/5.1/dataload/TensorLoader.lua:8: assertion failed!
stack traceback:
	[C]: in function 'assert'
	...eo/torch/install/share/lua/5.1/dataload/TensorLoader.lua:8: in function '__init'
	...cojavierarceo/torch/install/share/lua/5.1/torch/init.lua:91: in function <...cojavierarceo/torch/install/share/lua/5.1/torch/init.lua:87>
	[C]: in function 'TensorLoader'
	[string "dataloader = dl.TensorLoader(inputs, actions)..."]:1: in main chunk
	[C]: in function 'xpcall'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:210: in function <...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:174>
	...ojavierarceo/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	...ojavierarceo/torch

In [3]:
dataloader

In [ ]:
i = 1
for k, input, target, action in dataloader:sampleiter(1,5) do
   print(string.format("batch %d, nsampled = %d", i, k))
   i = i + 1
    if i==5 then
        print(input, target, action)
    end
end

In [ ]:
dl = require 'dataload'

In [ ]:
batchSize = 3

In [ ]:
dataloader = dl.TensorLoader(tst, tst)

In [ ]:
for i, inputs, targets in dataloader:sampleiter(batchSize, tst:size()) do
    print(i, inputs, targets)
end

In [ ]:
for i, inputs, targets in testSet:sampleiter(batchSize, testSet:size()) do
   trainInputs:resize(inputs:size()):copy(inputs)
   trainTargets:resize(targets:size()):copy(targets)
end

In [ ]:
tst:sampleiter(5,3)

In [ ]:
tst[indx]:resize(12, 4)

In [ ]:
torch.cat(summaryBatch, summaryBatch, 1)